In [ ]:
# from mrcnn.config import Config
# from mrcnn import model as modellib
# from mrcnn import visualize
# import mrcnn
# from mrcnn.utils import Dataset
# from mrcnn.model import MaskRCNN
# import numpy as np
# from numpy import zeros
# from numpy import asarray
# import colorsys
# import argparse
# import imutils
# import random
# import cv2
# import os
# import time
# from matplotlib import pyplot
# from matplotlib.patches import Rectangle
# from keras.models import load_model
# %matplotlib inline
# from os import listdir
# from xml.etree import ElementTree


import os
import sys
import cv2
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import skimage


# Import Mask RCNN
# sys.path.append(ROOT_DIR)  # To find local version of the library
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname("__file__")))+"/Mask-RCNN-with-custom-datasets/Mask_RCNN")
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

%matplotlib inline

random.seed(13)

In [ ]:
import coco
config = coco.CocoConfig()
COCO_DIR = os.path.abspath(r'../Mask-RCNN-with-custom-datasets')

ROOT_DIR = "../Mask-RCNN-with-custom-datasets/Mask_RCNN" 
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class InferenceConfig(coco.CocoConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
inference_config.display()

In [ ]:
class_names = ['stone']

folder_num = 1
folder_list = [i for i in range(folder_num)]

for one_folder in folder_list:
    # Load a random image from the images folder
    IMAGE_DIR = "/home/cai/Desktop/green/" + str(one_folder)

    try:
        os.mkdir("crop_dataset/%d" %one_folder)
    except:
        pass
    
#     file_names = next(os.walk(IMAGE_DIR))[2]
#     file_names.sort()

    # 3 light / 5 arr
#     file_list = file_names[10:15]
    file_list = os.listdir(IMAGE_DIR)

    crop_cnt = 0

    for one_img in file_list:
        sample_file_name = os.path.join(IMAGE_DIR, one_img)

        image = skimage.io.imread(sample_file_name)
        cv_image = cv2.imread(sample_file_name)
        # Run detection
        results = model.detect([image], verbose=1)
        # Visualize results
        r = results[0]
#         visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
#                                    ['BG', 'stone'], r['scores'])

#         plt.figure(figsize=(8,8))
#         plt.imshow(cv_image)
#         plt.show()
        masks = r["masks"]

        stones = np.array(np.where(masks==True))

        max_stone_num = np.max(stones[2])


        for idx in range(max_stone_num+1):


            one_stone = np.where(stones[2]==idx)
            one_stone_y = stones[0][one_stone]
            one_stone_x = stones[1][one_stone]
            
            
            
            

            min_x = np.min(one_stone_x)
            max_x = np.max(one_stone_x)

            min_y = np.min(one_stone_y)
            max_y = np.max(one_stone_y)

            width = max_x - min_x + 1
            height = max_y - min_y + 1

            one_stone_img = np.zeros((height,width,4))
            
            for x,y in list(zip(one_stone_x, one_stone_y)):
#                 one_stone_img[x-min_x][y-min_y][:] = cv_image[x][y][:]
#                 print(np.array(cv_image).shape)
                one_stone_img[y-min_y][x-min_x][:3] = image[y][x][:]
                one_stone_img[y-min_y][x-min_x][3] = 255
                
#             plt.figure(figsize=(8,8))
#             plt.imshow(one_stone_img.astype('uint8'))
#             plt.show()
            one_stone_img = (one_stone_img).astype(np.uint8)
            
            
        #     one_stone_img = cv2.cvtColor(one_stone_img, cv2.COLOR_BGR2BGRA)
            cv2.imwrite("crop_dataset/%d/%d.png" %(one_folder,crop_cnt), one_stone_img)

            crop_cnt+=1



In [ ]:
import numpy as np
import cv2
img = cv2.imread('...', 0) # Read in your image
# contours, _ = cv2.findContours(...) # Your call to find the contours using OpenCV 2.4.x
_, contours, _ = cv2.findContours(...) # Your call to find the contours
idx = ... # The index of the contour that surrounds your object
mask = np.zeros_like(img) # Create mask where white is what we want, black otherwise
cv2.drawContours(mask, contours, idx, 255, -1) # Draw filled contour in mask
out = np.zeros_like(img) # Extract out the object and place into output image
out[mask == 255] = img[mask == 255]

# Now crop
(y, x) = np.where(mask == 255)
(topy, topx) = (np.min(y), np.min(x))
(bottomy, bottomx) = (np.max(y), np.max(x))
out = out[topy:bottomy+1, topx:bottomx+1]

# Show the output image
cv2.imshow('Output', out)
cv2.waitKey(0)
cv2.destroyAllWindows()